Movie Renege
Covers:

Resources: Resource
Condition events
Shared events
This examples models a movie theater with one ticket counter selling tickets for three movies (next show only). People arrive at random times and try to buy a random number (1–6) of tickets for a random movie. When a movie is sold out, all people waiting to buy a ticket for that movie renege (leave the queue).

The movie theater is just a container for all the related data (movies, the counter, tickets left, collected data, …). The counter is a Resource with a capacity of one.

The moviegoer process starts waiting until either it’s his turn (it acquires the counter resource) or until the sold out signal is triggered. If the latter is the case it reneges (leaves the queue). If it gets to the counter, it tries to buy some tickets. This might not be successful, e.g. if the process tries to buy 5 tickets but only 3 are left. If less than two tickets are left after the ticket purchase, the sold out signal is triggered.

Moviegoers are generated by the customer arrivals process. It also chooses a movie and the number of tickets for the moviegoer.

"""
Movie renege example

Covers:

- Resources: Resource
- Condition events
- Shared events

Scenario:
  A movie theatre has one ticket counter selling tickets for three
  movies (next show only). When a movie is sold out, all people waiting
  to buy tickets for that movie renege (leave queue).

"""

In [80]:
import collections
import random

In [81]:
import simpy

In [82]:
RANDOM_SEED = 42
TICKETS = 50 # Number of tickets per movie
SIM_TIME = 120 # Simulate until

In [83]:
TIME_OUT = 0.5
NUMBER_TICKETS_RANDOM = [1, 6]

In [84]:
def moviegoer(env, movie, num_tickets, theater):
    """A moviegoer tries to by a number of tickets (*num_tickets*) for
    a certain *movie* in a *theater*.

    If the movie becomes sold out, she leaves the theater. If she gets
    to the counter, she tries to buy a number of tickets. If not enough
    tickets are left, she argues with the teller and leaves.

    If at most one ticket is left after the moviegoer bought her
    tickets, the *sold out* event for this movie is triggered causing
    all remaining moviegoers to leave.

    """
    with theater.counter.request() as my_turn:
        # Wait until its our turn or until the movie is sold out
        result = yield my_turn | theater.sold_out[movie]

        # Check if it's our turn or if movie is sold out
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            return

        # Check if enough tickets left.
        if theater.available[movie] < num_tickets:
            # Moviegoer leaves after some discussion
            yield env.timeout(TIME_OUT)
            return

        # Buy tickets
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Trigger the "sold out" event for the movie
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)

In [85]:
def customer_arrivals(env, theater):
    """Create new *moviegoers* until the sim time reaches 120."""
    while True:
        yield env.timeout(random.expovariate(1 / TIME_OUT))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(NUMBER_TICKETS_RANDOM[0], NUMBER_TICKETS_RANDOM[1])
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))

In [86]:
Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')

In [87]:
# Setup and start the simulation
print('Movie renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

Movie renege


In [88]:
# Create movie theater
counter = simpy.Resource(env, capacity=1)
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

In [89]:
print('Movies')
print(movies)
print('Available')
print(available)
print('Sold Out')
print(sold_out)
print('When sold out')
print(when_sold_out)
print('Number renegers')
print(num_renegers)
print('Theater')
print(theater)

Movies
['Python Unchained', 'Kill Process', 'Pulp Implementation']
Available
{'Python Unchained': 50, 'Kill Process': 50, 'Pulp Implementation': 50}
Sold Out
{'Python Unchained': <Event() object at 0x1076c3880>, 'Kill Process': <Event() object at 0x1076c37f0>, 'Pulp Implementation': <Event() object at 0x1076c3490>}
When sold out
{'Python Unchained': None, 'Kill Process': None, 'Pulp Implementation': None}
Number renegers
{'Python Unchained': 0, 'Kill Process': 0, 'Pulp Implementation': 0}
Theater
Theater(counter=<simpy.resources.resource.Resource object at 0x1076c3280>, movies=['Python Unchained', 'Kill Process', 'Pulp Implementation'], available={'Python Unchained': 50, 'Kill Process': 50, 'Pulp Implementation': 50}, sold_out={'Python Unchained': <Event() object at 0x1076c3880>, 'Kill Process': <Event() object at 0x1076c37f0>, 'Pulp Implementation': <Event() object at 0x1076c3490>}, when_sold_out={'Python Unchained': None, 'Kill Process': None, 'Pulp Implementation': None}, num_renege

In [90]:
# Start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

In [91]:
# Analysis/results
for movie in movies:
    if theater.sold_out[movie]:
        print('Movie "%s" sold out %.1f minutes after ticket counter '
              'opening.' % (movie, theater.when_sold_out[movie]))
        print('  Number of people leaving queue when film sold out: %s' %
              theater.num_renegers[movie])

Movie "Python Unchained" sold out 38.0 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 16
Movie "Kill Process" sold out 43.0 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 5
Movie "Pulp Implementation" sold out 28.0 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 5
